In [1]:
from opendp.trans import *
from opendp.meas import *
from opendp.core import *

enable_features("floating-point")

# establish data information
data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')
var_names = ["age", "sex", "educ", "race", "income", "married", "pid"]

# TODO: Remove column headers
with open(data_path) as input_data:
    data = input_data.read()

In [2]:
def _binary_search(predicate, start, end):
    if start > end:
        raise ValueError

    if not predicate(end):
        raise ValueError("no possible value in range")

    while True:
        mid = (start + end) / 2
        passes = predicate(mid)

        if passes and end - start < .00001:
            return mid

        if passes:
            end = mid
        else:
            start = mid

def check_scale(scale, preprocessor, dataset_distance, epsilon):
    """
    Return T/F
    :param scale:
    :param preprocessor:
    :param dataset_distance:
    :param epsilon:
    :return:
    """
    return (preprocessor >> make_base_laplace(scale)).check(dataset_distance, epsilon)


In [5]:
def make_dp_mean(col_names, index, lower, upper, n, epsilon):
    """
    Draft of a function to be used on the backend for DPCreator
    :param index: Column index to select data from
    :param data_type: Type to cast data to
    :param lower: Lower bound for clamp
    :param upper: Upper bound for clamp
    :param n: Estimated number of values in data
    :param epsilon: Privacy budget
    :return:
    """
    preprocessor = (
        # Convert data into Vec<Vec<String>>
        make_split_dataframe(separator=",", col_names=col_names) # >>
        # Selects a column of df, Vec<str>
        # make_select_column(key=index, T=str) # >>
        # Cast the column as Vec<Optional<Float>>
        # make_cast(TI=str, TO=float) # >>
        # Impute missing values to 0 Vec<Float>
        # make_impute_constant(0.) >>
        # Clamp age values
        # make_clamp(lower, upper) >>
        # make_resize_bounded(0., n, lower, upper) >>
        # make_bounded_mean(lower, upper, n=n, T=float)
    )
    scale = _binary_search(lambda s: check_scale(s, preprocessor, 1, epsilon), 0., 100.)
    preprocessor = preprocessor >> make_base_laplace(scale)
    return preprocessor(data)



In [6]:
epsilon = 1.
col_names = ["A","B","C","D","E"]
p = make_dp_mean(col_names, "A", 0., 1000000., 1000, epsilon)
res = p(data)
print(f"Epsilon: {epsilon}\nDP Mean: {res}")


OpenDPException: DomainMismatch("Intermediate domain mismatch")
	   0: backtrace::backtrace::libunwind::trace
	             at /Users/ethancowan/.cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.61/src/backtrace/libunwind.rs:90:5
	      backtrace::backtrace::trace_unsynchronized
	             at /Users/ethancowan/.cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.61/src/backtrace/mod.rs:66:5
	   1: backtrace::backtrace::trace
	             at /Users/ethancowan/.cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.61/src/backtrace/mod.rs:53:14
	   2: backtrace::capture::Backtrace::create
	             at /Users/ethancowan/.cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.61/src/capture.rs:176:9
	   3: backtrace::capture::Backtrace::new_unresolved
	             at /Users/ethancowan/.cargo/registry/src/github.com-1ecc6299db9ec823/backtrace-0.3.61/src/capture.rs:170:9
	   4: opendp::comb::chain::make_chain_mt
	             at /Users/ethancowan/Ide

In [ ]:
import opendp